In [35]:
import torch
# check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(f"Utilisation du périphérique : {device}")
torch.cuda.get_device_name()


Utilisation du périphérique : cpu


'NVIDIA GeForce RTX 3070 Laptop GPU'

# Chargement du dataset

In [36]:
from datasets import load_dataset

# dataset = load_dataset("bigcode/the-stack-smol-xl", data_dir="data/dockerfile")
dataset = load_dataset("bigcode/the-stack-smol-xl", data_dir="data/python", split="train[:20]")

dataset

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 20
})

# Chargement du model


In [37]:
from opco_4_utils import init_model, do_predict

tokenizer, model = init_model("Salesforce/codegen-350M-mono")




Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [38]:
# On ne conserve que le code car rien d'autre interessant dans le dataset

dataset = dataset.map(
    lambda ex: {
        'code': ex['content']
    },
    remove_columns=dataset.column_names
)

dataset

Dataset({
    features: ['code'],
    num_rows: 20
})

Le dataset est préparer

il faut maintenant 'tokeniser' les lignes

In [39]:
tokenised_dataset = dataset.map(
    lambda ex: tokenizer(ex['code'])
)

tokenised_dataset

Dataset({
    features: ['code', 'input_ids', 'attention_mask'],
    num_rows: 20
})

In [42]:
import os
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Définir le répertoire de sortie pour le modèle fine-tuné
output_dir = os.path.join("export_model", "./codegen-350m-mono-finetuned-python")

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=1,  # Un seul epoch peut déjà donner de bons résultats
    per_device_train_batch_size=4,  # Réduire si vous avez des erreurs de mémoire (OOM)
    gradient_accumulation_steps=2, # Simule un batch size plus grand (4*2=8)
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=2e-5,
    fp16=True,  # Utiliser le mixed precision pour accélérer l'entraînement (nécessite un GPU compatible)
    logging_steps=500,
#    report_to="tensorboard", # Optionnel: pour visualiser les logs
)


In [43]:
# Le Data Collator prépare les batches pour le language modeling
# mlm=False car c'est du Causal LM (style GPT), pas du Masked LM (style BERT)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Lancer le fine-tuning 🚀
trainer.train()

# Sauvegarder le modèle final
trainer.save_model(output_dir)
print(f"Modèle sauvegardé dans {output_dir}")

/tmp/ipykernel_27131/411985426.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.